In [1]:
import os
import numpy as np
import pandas as pd
from main import *
from utilities import *
from parseSources import *

In [2]:
source_data =  parse_sources()
source_data

,County,Year,Event,Location,Source:1,Source:2,Source:3,Source:4,Latitude,Longitude
0,piscataquis,1912,piscataquis central fair,dover-foxcroft,piscataquis observer|1912-10-03 00:00:00|premi...,NaN,NaN,NaN,45.186827,-69.232015
1,piscataquis,1894,piscataquis central fair,dover-foxcroft,piscataquis observer|1894-09-27 00:00:00|premi...,NaN,NaN,NaN,45.186827,-69.232015
2,piscataquis,1892,piscataquis central fair,dover-foxcroft,maine farmer|1892-10-13 00:00:00|premiums||1|,piscataquis observer|1892-09-29 00:00:00|premi...,NaN,NaN,45.186827,-69.232015
3,piscataquis,1884,piscataquis central fair,dover-foxcroft,piscataquis observer|1884-10-09 00:00:00|premi...,NaN,NaN,NaN,45.186827,-69.232015
4,piscataquis,1887,piscataquis central fair,,piscataquis observer|1887-09-29 00:00:00||desc...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1139,waldo,1910,north waldo and penobscot fair,monroe,republican journal|1910-09-22 00:00:00|premium...,NaN,NaN,NaN,44.616108,-69.021310
1140,waldo,1914,north waldo and penobscot fair,monroe,republican journal|1914-09-17 00:00:00|premium...,NaN,NaN,NaN,44.616108,-69.021310
1141,waldo,1915,north waldo and penobscot fair,monroe,republican journal|1915-09-16 00:00:00|premium...,NaN,NaN,NaN,44.616108,-69.021310
1142,waldo,1911,north waldo and penobscot fair,monroe,republican journal|1911-09-28 00:00:00|premium...,NaN,NaN,NaN,44.616108,-69.021310
